In [6]:
from torchvision import utils
from data_loader import *
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from PIL import Image
from train_utils import *
import pickle as pkl
import time
import itertools
import matplotlib.pyplot as plt
import json
import random
import os
import csv
import torch.nn as nn
np.random.seed(42)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
# import argparse
# parser = argparse.ArgumentParser(description='Choose a config file')

# # experiment
# parser.add_argument(
#     '--experiment',
#     default='dummy',
#     help='Choose a config file (default: \'base\')'
# )


# args = parser.parse_args()

# # grab values from arguments
# experiment = args.experiment

In [3]:
# if experiment == "base1":
#     import base1_config as cfg
#     from base_model import BaseCaptioner as Model
# else:
#     import dummy_config as cfg
#     from base_model import BaseCaptioner as Model

In [4]:
# cfg = cfg.cfg
from base_model import BaseModel as Model
batch_size = 5
valid_batch_size = 20
num_workers = 1
epochs = 1
model_type = "dummy"



In [8]:
# tr5534_data = pkl.load(open("TR5534.pkl", "rb"))
cb513_data = json.load(open("CB513.json", "r"))

In [9]:
len_train = len(cb513_data)
percent_train = .8

train_start = 0
train_end = int(len_train * percent_train)

val_start = train_end
val_end = len_train

In [10]:
ids = np.random.choice(len_train, len_train, replace=False)

In [11]:
len_train

514

In [12]:
train_loader, len_train = get_loader(protein_data=cb513_data,
                                     ids=[0, 2],
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=num_workers)

val_loader, len_val = get_loader(protein_data=cb513_data,
                                 ids=[0, 2],
                                 batch_size=batch_size,
                                 shuffle=True,
                                 num_workers=num_workers)

test_loader, len_test = get_loader(protein_data=cb513_data,
                                   ids=[0, 2],
                                   batch_size=batch_size,
                                   shuffle=True,
                                   num_workers=num_workers)

print(len_train, len_val, len_test)

Loading 0/2 proteins
2 2 2
Loading 0/2 proteins
2 2 2
Loading 0/2 proteins
2 2 2
2 2 2


In [13]:
latest_model_path = os.path.join(model_type, "latest_model.pt")
best_model_path = os.path.join(model_type, "best_model.pt")
optim_path = os.path.join(model_type, "optim.pt")
stats_path = os.path.join(model_type, "stats.pkl")

def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init._kaiming_(m.weight.data)


        # ??? https://github.com/pytorch/pytorch/issues/3418
        if m.bias is not None:
            m.bias.data.zero_()
            # torch.nn.init.xavier_uniform(m.bias.data)
            
criterion = nn.CrossEntropyLoss()

In [14]:
cnn_1d_3 = nn.Conv1d(in_channels=51, out_channels=100, stride=1, kernel_size=3, padding=1, bias=True).cuda()
cnn_1d_5 = nn.Conv1d(in_channels=51, out_channels=100, stride=1, kernel_size=5, padding=2, bias=True).cuda()
gru_f_1 = nn.GRU(input_size=251, hidden_size=250, num_layers=1, batch_first=True).cuda()
gru_b_1 = nn.GRU(input_size=251, hidden_size=250, num_layers=1, batch_first=True).cuda()

gru_f_2 = nn.GRU(input_size=500, hidden_size=500, num_layers=1, batch_first=True).cuda()
gru_b_2 = nn.GRU(input_size=500, hidden_size=500, num_layers=1, batch_first=True).cuda()

gru_f_3 = nn.GRU(input_size=500, hidden_size=500, num_layers=1, batch_first=True).cuda()
gru_b_3 = nn.GRU(input_size=500, hidden_size=500, num_layers=1, batch_first=True).cuda()

cnn_1d_1_1 = nn.Conv1d(in_channels=751, out_channels=500, stride=1, kernel_size=1, bias=True).cuda()
cnn_1d_1_2 = nn.Conv1d(in_channels=1000, out_channels=500, stride=1, kernel_size=1, bias=True).cuda()

fc1 = nn.Linear(500, 1024).cuda()
fc2 = nn.Linear(1024, 9).cuda()

embedding = nn.Embedding(22, 22).cuda()


In [15]:
def forward(x):
    one_hot = x[:, 0:22, :].argmax(axis=1)
    embedded = embedding(one_hot.long()).permute(0, 2, 1)
    x[:, 0:22, :] = embedded
    
    local_block_3 = cnn_1d_3(x)
    local_block_5 = cnn_1d_5(x)
    x = nn.functional.relu(torch.cat((x, local_block_3, local_block_5), dim=1))
    x = x.permute(0, 2, 1)
    
    
    T = x.shape[1]
    h_t_f = torch.zeros(1, x.shape[0], 250).cuda()
    h_t_b = torch.zeros(1, x.shape[0], 250).cuda()

    h_f = []
    h_b = []
    
    
    for t in range(T):
        input_t_f = x[:, t, :].unsqueeze(1)
        input_t_b = x[:, t, :].unsqueeze(1)
        
        _, h_t_f = gru_f_1(input_t_f, h_t_f)
        _, h_t_b = gru_b_1(input_t_b, h_t_b)

        h_f.append(h_t_f)
        h_b.append(h_t_b)


    F = torch.cat(h_f, dim=2)
    B = torch.cat(h_b, dim=2)
    O1 = torch.cat((F, B), dim=2).view((x.shape[0], 700, -1))
    
    
    x = torch.cat((x, O1), dim=2)

    x = nn.functional.relu(cnn_1d_1_1(x.view([-1, 751, 700])))


    h_t_f = torch.zeros(1, x.shape[0], 500).cuda()
    h_t_b = torch.zeros(1, x.shape[0], 500).cuda()

    h_f = []
    h_b = []

    x = x.permute([0, 2, 1])
    T = x.shape[1]
    for t in range(T):
        input_t_f = x[:, t, :].unsqueeze(1)
        input_t_b = x[:, t, :].unsqueeze(1)


        _, h_t_f = gru_f_2(input_t_f, h_t_f)
        _, h_t_b = gru_b_2(input_t_b, h_t_b)

        h_f.append(h_t_f)
        h_b.append(h_t_b)

    F = torch.cat(h_f, dim=2)
    B = torch.cat(h_b, dim=2, )
    O2 = (F + B).view((x.shape[0], 700, -1))
    

    x = torch.cat((x, O2), dim=2).permute(0, 2, 1)
    x = nn.functional.relu(cnn_1d_1_2(x))
    

    h_t_f = torch.zeros(1, x.shape[0], 500).cuda()
    h_t_b = torch.zeros(1, x.shape[0], 500).cuda()

    h_f = []
    h_b = []
    
    x = x.permute([0, 2, 1])
    T = x.shape[1]
    
    for t in range(T):
        input_t_f = x[:, t, :].unsqueeze(1)
        input_t_b = x[:, t, :].unsqueeze(1)


        _, h_t_f = gru_f_3(input_t_f, h_t_f)
        _, h_t_b = gru_b_3(input_t_b, h_t_b)

        h_f.append(h_t_f)
        h_b.append(h_t_b)
        
    F = torch.cat(h_f, dim=2)
    B = torch.cat(h_b, dim=2)
    x = (F + B).view([x.shape[0], 700, 500])
    
    x = fc1(x)
    x = nn.functional.relu(x)
    x = fc2(x)
    
    return x

    

In [37]:
all_labels = []
all_preds = []
for iter, (X, Y, seq_lens) in enumerate(train_loader):
    seq_lens = seq_lens.numpy()
    x = X.reshape([-1, 700, 51]).cuda()
    x = x.permute(0, 2, 1)
    print(x[0, 0:22, 0])
    Y = Y.view([-1, 700, 9])
    print(Y[0:5])
    print(Y[0:5].max(axis=2))
 
    break

    out = forward(x)
    print(Y.shape)
    targets = Y.argmax(dim=1).long().cuda()
    print(out.shape, Y.shape)
#     loss = criterion(out, targets)
    labels = Y.argmax(dim=2).cpu().numpy()
    print(labels.shape)
    predictions = out.argmax(axis=2).cpu().numpy()
    print(out.shape)
    
    
    for label, prediction, length in zip(labels, predictions, seq_lens):
        all_labels += list(label[:length])
        all_preds += list(prediction[:length])
        
    print(len(all_labels), sum(seq_lens), len(all_preds))
    break

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0.], device='cuda:0')
tensor([[[1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1]],

        [[1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1]]], dtype=torch.int32)
torch.return_types.max(
values=tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32),
indices=tensor([[0, 0, 0,  ..., 8, 8, 8],
        [0, 0, 0,  ..., 8, 8, 8]]))


In [17]:
X = torch.ones((20, 50, 1))

In [ ]:
X_hot = X[:, 0:21, 0].argmax(axis=1)

In [ ]:
X_hot.shape

In [ ]:
# model = Model().cuda()
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.0004)

In [ ]:
# stats_dict, model = train(1, model, stats_path, train_loader, val_loader, optimizer, criterion,
#                           len_train, len_val, latest_model_path, best_model_path, optim_path)